In [1]:
import os
print(os.getcwd())
os.chdir('/oak/stanford/groups/whwong/group_scratch/tdaley/CS231N/OrganoidNet/models/')
print(os.getcwd())

/home/users/tdaley
/oak/stanford/groups/whwong/group_scratch/tdaley/CS231N/OrganoidNet/models


In [2]:
from skimage import io, transform, color
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
from torch.utils import data
import numpy as np
import sys
import pandas as pd
import math
import torchvision.models as models
import dataLoader
import matplotlib.pyplot as plt

In [3]:
#model = torch.load('classification_model.pth', map_location='cpu')
#class TempModel(nn.Module):
#    def __init__(self):
#        self.conv1 = nn.Conv2d(3, 5, (3, 3))
#    def forward(self, inp):
#        return self.conv1(inp)

#model = TempModel()
#model.load_state_dict(torch.load('classification_model.pth', map_location='cpu'))
#model.eval()


resnet18 = models.resnet18()
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('classification_checkpoint.pth', map_location='cpu')['state_dict'])
model.eval()


/share/software/user/open/py-pytorch/1.0.0_py36/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/share/software/user/open/py-pytorch/1.0.0_py36/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/share/software/user/open/py-pytorch/1.0.0_py36/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchN

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [4]:
class OrganoidMultipleDataset(data.Dataset):
    'dataset class for microwell organoid images'
    def __init__(self, path2files, image_names, Y, mean_sd_dict, transforms=None):
        for k, image_name in image_names.items():
            assert len(image_name) == len(Y)
        self.path = path2files
        self.image_names = image_names
        self.Y = Y
        self.mean_sd_dict = mean_sd_dict
        self.transforms = transforms
    def __len__(self):
        return len(self.Y)
    def getXimage(self, index):
        all_images_list = []
        for day,img_names in self.image_names.items():
            #print(day, "   ", index)
            
            img_name = img_names[index]
            img_loc = os.path.join(self.path, img_name)
            image = io.imread(img_loc)
            mean, sd = self.mean_sd_dict[day]
            image = np.true_divide(color.rgb2gray(image) - mean, sd)
            all_images_list.append(image)
        images = np.array(all_images_list)
        return torch.from_numpy(images).float()
    def getY(self, index):
        Y = self.Y[index]
        return torch.from_numpy(np.asarray(self.Y[index], dtype=float)).float()
    def __getitem__(self, index):
        X = self.getXimage(index)
        y = self.getY(index)
        if self.transforms is not None:
            X = self.transforms(X)
        return X, y 

In [5]:
test_labels = pd.read_csv('../data_description/A1_A2_C1_filtered_test_v2.csv')
print(test_labels.head())
print(test_labels.shape)
test_image_names = {2:test_labels['image_name_2'],8:test_labels['image_name_8'],5:test_labels['image_name_5']}
test_y = test_labels['has_cell_13']
print(test_y.shape)

   Unnamed: 0 condition  well_id  day_0  well_label  \
0        7328        A1      335      0         335   
1        7329        A1     1105      0        1105   
2        7330        A1     3397      0        3397   
3        7331        A1     1684      0        1684   
4        7332        C1     1077      0        1077   

                      image_name_0  has_cell_0  hyst2_area_0  day_1  \
0  well_A1/well0335_day00_well.png           1          2260      1   
1  well_A1/well1105_day00_well.png           1          2104      1   
2  well_A1/well3397_day00_well.png           1            75      1   
3  well_A1/well1684_day00_well.png           0             0      1   
4  well_C1/well1077_day00_well.png           0             0      1   

                      image_name_1      ...        has_cell_11  hyst2_area_11  \
0  well_A1/well0335_day01_well.png      ...                  0              0   
1  well_A1/well1105_day01_well.png      ...                  0              0   

In [6]:
mean_sd_dict = {2: [0.49439774802337344, 0.16087996922691195],
 8: [0.5177020917650417, 0.15714445907773483],
 5: [0.5013496452715945, 0.1605951051365687],              }

In [7]:
path = '../data/14day/'
test_set = OrganoidMultipleDataset(path2files = path, 
                                   image_names = test_image_names, 
                                   Y = test_labels['has_cell_13'],
                                   mean_sd_dict=mean_sd_dict)

In [8]:
params = {'batch_size': test_labels.shape[0], # all validation data
          'shuffle': False, 'num_workers' : 1}
test_generator = data.DataLoader(test_set, **params)


In [9]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

device = torch.device("cpu")
test_predictions = np.zeros(test_y.shape)
test_values = np.zeros(test_y.shape)
with torch.no_grad():
    for x, y in test_generator:
        x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)
        scores = model(x)
        _, preds = scores.max(1)
        test_values = (y.data).cpu().numpy()
        test_predictions = (preds.data).cpu().numpy()



In [29]:
print(test_predictions.shape)
print(test_values.shape)
print(test_predictions[0:4])
print(test_values[0:4])
print(test_y[0:4])
print(sum(test_predictions == test_values)/test_predictions.shape[0]) # accuracy
confusion_matrix = np.zeros((2, 2))
confusion_matrix[0, 0] = sum(np.logical_and(test_predictions == 0, test_values == 0))/test_predictions.shape[0]
confusion_matrix[1, 0] = sum(np.logical_and(test_predictions == 1, test_values == 0))/test_predictions.shape[0]
confusion_matrix[0, 1] = sum(np.logical_and(test_predictions == 0, test_values == 1))/test_predictions.shape[0]
confusion_matrix[1, 1] = sum(np.logical_and(test_predictions == 1, test_values == 1))/test_predictions.shape[0]
print(confusion_matrix)
np.savetxt("test_set_confusion_matrix.txt", confusion_matrix, delimiter = '\t')

np.savetxt("test_set_predictions.txt", test_predictions, delimiter = '\t')
np.savetxt("test_set_y.txt", test_values, delimeter = '\t')
#import seaborn
#seaborn.heatmap(df_cm, annot=True, cmap = seaborn.color_palette("BuGn_r"))

(815,)
(815,)
[0 0 0 1]
[1 1 0 1]
0    1
1    1
2    0
3    1
Name: has_cell_13, dtype: int64
0.7705521472392638
[[0.28343558 0.0993865 ]
 [0.13006135 0.48711656]]


TypeError: savetxt() got an unexpected keyword argument 'delimeter'

In [21]:
from sklearn.manifold import TSNE
def flatten(x):
    return x.view(-1)  # "flatten" the C * H * W values into a single vector per image

X = np.zeros((test_labels.shape[0], 3*193*193))
for i in range(test_labels.shape[0]):
    X[i,] = flatten(test_set.getXimage(i)).numpy()

In [24]:
tsne = TSNE(n_components=2, verbose=1, perplexity=30, n_iter=300)
tsne_results = tsne.fit_transform(X)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 815 samples in 3.318s...
[t-SNE] Computed neighbors for 815 samples in 118.524s...
[t-SNE] Computed conditional probabilities for sample 815 / 815
[t-SNE] Mean sigma: 110.189131
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.075996
[t-SNE] Error after 300 iterations: 1.045783


In [30]:
print(tsne_results.shape)
print(tsne_results[0:4])
np.savetxt('test_set_tsne_embedding.txt', tsne_results, delimiter = '\t')

(815, 2)
[[-11.483752   -9.282276 ]
 [ -7.5917945   6.7265086]
 [ -5.707936   -2.2783551]
 [ -8.738314   -8.937672 ]]
